In [ ]:
#Import necessary libraries

import pandas as pd
from sqlalchemy import create_engine
from psycopg2.extras import execute_batch
import psycopg2
import requests
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
import datetime
import re
from serpapi import GoogleSearch
from datasets import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

In [2]:
#Connecting to the database

load_dotenv()
host = 'localhost'
port = 5432 
user = 'postgres'
password = os.getenv('PASSWORD_DB')
db = 'postgres' 
conn = psycopg2.connect(host=host, port=port, user=user, password=password, dbname=db)
connection_string = f"postgresql://{user}:{password}@{host}/{db}"
engine = create_engine(connection_string)


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Scrape news from CoinDesk

options = webdriver.ChromeOptions()


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
links = []
xpaths = []

for i in range(371):
    base_url = f'https://www.coindesk.com/tag/ethereum/{i}/'
    driver.get(base_url)
    print(i)

    for i in range(1,11):
        xpaths.append(f'//*[@id="fusion-app"]/div/div/div/main/div[2]/div/div[1]/div[{i}]') 

    for xpath in xpaths:
        try:
            articles_container = driver.find_element(By.XPATH, xpath)
        
            article_elements = articles_container.find_elements(By.XPATH, './/a')
        
            for article in article_elements:
                href = article.get_attribute('href')
                if href and href.startswith("https://www.coindesk.com/"):
                    links.append(href)
        except Exception as e:
            print(f"Error finding elements with XPath {xpath}: {e}")

    for i, link in enumerate(links, start=1):
        print(f"Link {i}: {link}")

    xpaths.clear()



driver.quit()

In [ ]:
def has_date(url):
    return bool(re.search(r'\d{4}/\d{2}/\d{2}', url))

links_step_2 = [url for url in links if has_date(url)]
links_end = list(set(links_step_2))
links_end

links_df= pd.DataFrame(links_end)
links_df.to_sql(name = 'links_df'
            , con = engine
            , index = False
            , if_exists ='replace')
links_df

In [ ]:
links_dic = {"text": [], "time": []}

def get_content(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')


        base_content = soup.find(attrs={"data-module-name": "article-body-no-right-rail"})
        if base_content:
            paragraphs = base_content.find_all('p')
            all_text = "\n".join(paragraph.get_text() for paragraph in paragraphs)
        else:
            all_text = "Content not found"


        time_element = soup.find(class_="iOUkmj")
        time = time_element.get_text() if time_element else "Time not found"

        links_dic['text'].append(all_text)
        links_dic["time"].append(time)

    except Exception as e:
        print(f"Error fetching content from {link}: {e}")

def get_contents(page_links):
    for page_link in tqdm(page_links, desc="Fetching content"):
        get_content(page_link)


get_contents(links_end)


news = pd.DataFrame(links_dic)
news

In [ ]:
# Cleaning and correcting data fetched from CoinDesk

news = news[~news['text'].str.contains('Error fetching content from')]

def convert_date(date_str):
    date_str_cleaned = date_str.split(' UTC')[0].strip()
    
    date_str_cleaned = date_str_cleaned.replace('p.m.', 'PM').replace('a.m.', 'AM')
    
    formats = [
        '%b %d, %Y at %I:%M %p',
        '%b %d, %Y at %H:%M %p'
    ]
    
    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str_cleaned, fmt)
            return date_obj.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            continue
    return None

news['time'] = news['time'].apply(convert_date)

news.dropna(inplace=True)

news['time'] = pd.to_datetime(news['time'], format='%Y-%m-%d %H:%M:%S')

news

In [ ]:
# Fetch news data from NewsAPI, process it to extract relevant information, and clean the data

API_KEY = os.getenv('API_KEY_News_API')
BASE_URL = os.getenv('BASE_URL_News_API')

def fetch_news(query, from_date, to_date):
    params = {
        'q': query,
        'from': from_date,
        'to': to_date,
        'apiKey': API_KEY,
        'language': 'en'
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        return articles
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return []

def save_news_to_csv(articles, filename):
    df = pd.DataFrame(articles)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} articles to {filename}")

current_date = datetime.datetime.now()
from_date = (current_date - timedelta(days=30)).strftime('%Y-%m-%d')
to_date = (current_date).strftime('%Y-%m-%d')
articles = fetch_news('Ethereum', from_date, to_date)
news_api = pd.DataFrame(articles)
news_api


In [ ]:
# Fetch news from Google News


def fetch_all_ethereum_news(api_key, query="Ethereum", results_per_page=10):
    articles = []
    start = 0
    
    while True:
        params = {
            "engine": "google",
            "q": query,
            "tbm": "nws",
            "api_key": api_key,
            "num": results_per_page,
            "start": start
        }
        
        search = GoogleSearch(params)
        results = search.get_dict()
        
        news_results = results.get("news_results", [])
        if not news_results:
            break
        
        for news in news_results:
            article = {
                "title": news.get("title"),
                "snippet": news.get("snippet"),
                "date": news.get("date"),
                "link": news.get("link")
            }
            articles.append(article)
        
        start += results_per_page
    
    return pd.DataFrame(articles)


api_key = os.getenv('api_key_google')
news_df = fetch_all_ethereum_news(api_key)
news_df


In [13]:
# Convert the publication time from Google News to a standard datetime format

from datetime import datetime, timedelta

def convert_relative_time(time_str):

    hour_pattern = re.compile(r'(\d+)\s*hour')
    minute_pattern = re.compile(r'(\d+)\s*minute')
    day_pattern = re.compile(r'(\d+)\s*day')
    week_pattern = re.compile(r'(\d+)\s*week')
    month_pattern = re.compile(r'(\d+)\s*month')

    hours_match = hour_pattern.search(time_str)
    minutes_match = minute_pattern.search(time_str)
    days_match = day_pattern.search(time_str)
    weeks_match = week_pattern.search(time_str)
    months_match = month_pattern.search(time_str)

    hours = int(hours_match.group(1)) if hours_match else 0
    minutes = int(minutes_match.group(1)) if minutes_match else 0
    days = int(days_match.group(1)) if days_match else 0
    weeks = int(weeks_match.group(1)) if weeks_match else 0
    months = int(months_match.group(1)) if months_match else 0


    if hours > 0 or minutes > 0 or days > 0 or weeks > 0 or months > 0:
        absolute_time = datetime.now() - timedelta(
            hours=hours,
            minutes=minutes,
            days=days + weeks * 7
        ) - timedelta(days=months * 30)
        return absolute_time.strftime('%Y-%m-%d %H:%M:%S')
    
    return time_str



news_df['date'] = news_df['date'].apply(convert_relative_time)

news_df


In [15]:
from datetime import datetime, timedelta
def remove_newlines(text):
    return text.replace('\n', ' ')

news_df['snippet'] = news_df['snippet'].apply(remove_newlines)

def convert_date(date_str):
    for fmt in ('%Y-%m-%d %H:%M:%S', '%b %d, %Y'):
        try:
            return datetime.strptime(date_str, fmt).strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            continue
    return None

news_df['date'] = news_df['date'].apply(convert_date)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Fetch OHLC price data from Binance API at minute intervals

import datetime

API_KEY = os.getenv('API_KEY_OHLC')
BASE_URL = os.getenv('BASE_URL_OHLC')

def get_binance_ohlc(symbol, interval, start_time, end_time):
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': int(start_time.timestamp() * 1000),
        'endTime': int(end_time.timestamp() * 1000),
        'limit': 1000
    }
    response = requests.get(BASE_URL, params=params, headers={'X-MBX-APIKEY': API_KEY})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def format_data(data):
    formatted_data = []
    for item in data:
        timestamp = item[0] / 1000
        date_time = datetime.datetime.fromtimestamp(timestamp)
        formatted_data.append({
            'datetime': date_time,
            'open': float(item[1]),
            'high': float(item[2]),
            'low': float(item[3]),
            'close': float(item[4])
        })
    return formatted_data

time = []
end_date = datetime.datetime.now()
start_date = news['time'].min()

while end_date > start_date:
    start_interval = end_date - datetime.timedelta(minutes=1000)
    data = get_binance_ohlc('ETHUSDT', '1m', start_interval, end_date)
    
    if data:
        formatted_data = format_data(data)
        time.extend(formatted_data)
    
    end_date = start_interval

ohlc_df = pd.DataFrame(time)
ohlc_df

In [4]:
# Fetch OHLC price data from Binance API at hourly intervals

API_KEY = os.getenv('API_KEY_OHLC')
BASE_URL = os.getenv('BASE_URL_OHLC')

def get_binance_ohlc(symbol, interval, start_time, end_time):
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': int(start_time.timestamp() * 1000),
        'endTime': int(end_time.timestamp() * 1000),
        'limit': 10000
    }
    response = requests.get(BASE_URL, params=params, headers={'X-MBX-APIKEY': API_KEY})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def format_data(data):
    formatted_data = []
    for item in data:
        timestamp = item[0] / 1000
        date_time = datetime.datetime.fromtimestamp(timestamp)
        formatted_data.append({
            'datetime': date_time,
            'open': float(item[1]),
            'high': float(item[2]),
            'low': float(item[3]),
            'close': float(item[4])
        })
    return formatted_data


end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=40)

data = get_binance_ohlc('ETHUSDT', '1h', start_date, end_date)

if data:
    formatted_data = format_data(data)
    
    ohlc_df_h = pd.DataFrame(formatted_data)
else:
    print("No data available or error in response.")


In [ ]:
# Fetch OHLC price data from Binance API at daily intervals

API_KEY = os.getenv('API_KEY_OHLC')
BASE_URL = os.getenv('BASE_URL_OHLC')

def get_binance_ohlc(symbol, interval, start_time, end_time):
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': int(start_time.timestamp() * 1000),
        'endTime': int(end_time.timestamp() * 1000),
        'limit': 10000
    }
    response = requests.get(BASE_URL, params=params, headers={'X-MBX-APIKEY': API_KEY})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def format_data(data):
    formatted_data = []
    for item in data:
        timestamp = item[0] / 1000
        date_time = datetime.datetime.fromtimestamp(timestamp)
        formatted_data.append({
            'datetime': date_time,
            'open': float(item[1]),
            'high': float(item[2]),
            'low': float(item[3]),
            'close': float(item[4])
        })
    return formatted_data


end_date = datetime.datetime.now() - datetime.timedelta(days=41)
start_date = end_date - datetime.timedelta(days=720)

data = get_binance_ohlc('ETHUSDT', '1d', start_date, end_date)

if data:
    formatted_data = format_data(data)
    
    ohlc_df_d = pd.DataFrame(formatted_data)
else:
    print("No data available or error in response.")


In [ ]:
ohlc_df_d['datetime'] = pd.to_datetime(ohlc_df_d['datetime'])
ohlc_df_d['datetime'] = ohlc_df_d['datetime'] - pd.Timedelta(hours=4)

In [ ]:
ohlc_df.to_sql(name = 'df_ohlc'
            , con = engine
            , index = False
            , if_exists ='replace')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
# Rename columns in the DataFrame
news.rename(columns={"time": "timestamp"}, inplace=True)
ohlc_df.rename(columns={"datetime": "timestamp"}, inplace=True)

In [ ]:
news

In [ ]:
ohlc_df

In [20]:
# Merge datasets based on the 'timestamp' column

merged_df_1 = pd.merge_asof(news.sort_values('timestamp'), 
                          ohlc_df.sort_values('timestamp'), 
                          on='timestamp', 
                          direction='forward')

In [ ]:
merged_df_1

In [ ]:
# Add a label to the dataset to indicate which news articles are associated with price changes

merged_df_1['price_difference'] = merged_df_1['close'] - merged_df_1['open']
merged_df_1['label'] = (merged_df_1['price_difference'] > 0).astype(int)
merged_df_1

In [23]:
merged_df_1.rename(columns={"text":"content"},inplace=True)

In [ ]:
# Convert or adjust the time data to ensure consistency and alignment with other datasets

ohlc_df = pd.concat([ohlc_df_h, ohlc_df_d], ignore_index=True)
ohlc_df['timestamp'] = pd.to_datetime(ohlc_df['datetime'])
ohlc_df['timestamp'] = ohlc_df['timestamp'].dt.tz_localize(None)
ohlc_df.drop(columns="datetime",inplace=True)
ohlc_df

In [ ]:
eth_news_0['timestamp'] = pd.to_datetime(eth_news_0['publishedAt'])
eth_news_0['timestamp'] = eth_news_0['timestamp'].dt.round('H')
eth_news_0['timestamp'] = eth_news_0['timestamp'].dt.tz_localize(None)
eth_news_0 = eth_news_0[["timestamp","content"]]

In [ ]:
eth_news_1['timestamp'] = pd.to_datetime(eth_news_1['date'])
eth_news_1['timestamp'] = eth_news_1['timestamp'].dt.round('H')
eth_news_1['timestamp'] = eth_news_1['timestamp'].dt.tz_localize(None)
eth_news_1.rename(columns={'snippet': 'content'},inplace=True)
eth_news_1 = eth_news_1[["timestamp","content"]]

In [ ]:
eth_news_2['timestamp'] = pd.to_datetime(eth_news_2['date'])
eth_news_2['timestamp'] = eth_news_2['timestamp'].dt.round('H')
eth_news_2['timestamp'] = eth_news_2['timestamp'].dt.tz_localize(None)
eth_news_2.rename(columns={'snippet': 'content'},inplace=True)
eth_news_2 = eth_news_2[["timestamp","content"]]

In [ ]:
eth_news = pd.concat([eth_news_0, eth_news_1,eth_news_2], ignore_index=True)
eth_news.dropna(inplace=True)
eth_news

In [29]:
# Merge datasets based on the 'timestamp' column

merged_df_2 = pd.merge_asof(eth_news.sort_values('timestamp'), 
                          ohlc_df.sort_values('timestamp'), 
                          on='timestamp', 
                          direction='forward')

In [ ]:
# Add a label to the dataset to indicate which news articles are associated with price changes

merged_df_2['price_difference'] = merged_df_2['close'] - merged_df_2['open']
merged_df_2['label'] = (merged_df_2['price_difference'] > 0).astype(int)
merged_df_2

In [ ]:
# Concatenate two merged datasets to create a unified dataset

df = pd.concat([merged_df_1,merged_df_2],ignore_index=True)
df

In [ ]:
# Store the final dataset in the database

df.to_sql(name = 'df_teleg'
            , con = engine
            , index = False
            , if_exists ='replace')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
# Function to tokenize news articles into individual words or tokens

news_texts = df['content'].tolist()
labels = df['label'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [ ]:
# Tokenize news articles to split text into individual words or tokens

train_texts, val_texts, train_labels, val_labels = train_test_split(news_texts, labels, test_size=0.1)


train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})


train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column('label', 'labels')
val_dataset = val_dataset.rename_column('label', 'labels')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Import the model architecture for training

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
# Set up training arguments and initialize the trainer for model training

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

def compute_metrics(p):
    logits = p.predictions
    labels = p.label_ids
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
# Train the model using the training data and evaluate its performance on the validation set

trainer.train()
eval_results = trainer.evaluate()

In [ ]:
# Save the trained model to a file for future use or deployment

trainer.save_model('./final_model')
tokenizer.save_pretrained('./final_model')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------